- Check for Colab Env
> if we are in colab env we need to get the data + scripts from github or somewhere

In [ ]:
import os
if "COLAB_GPU" in os.environ:
    _useMultiProcessing = False
    os.system("git clone https://github.com/zaxo7/UNet-Playground.git")
    os.chdir("/content/UNet-Playground/")
else:
    _useMultiProcessing = True


- enable module autoreload and fix an import bug

In [11]:
%load_ext autoreload
%autoreload 2

import sys,os
sys.path.append(os.getcwd() + '/scripts')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- import the scripts and data

In [12]:
from scripts import data, model
import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

- load the dataset

- slice each image to small samples of size 188*188
- imgs -> rgb image
- mask -> mask
- edge -> edge_mask

In [ ]:
train_img_files = glob.glob('data/train/*.jpg')
test_img_files = glob.glob('data/test/*.jpg')

train = model.generate_train_dataset(train_img_files)

train_generator = model.generate_train_dataset(train_img_files)

train_set = model.generate_train_dataset_tf(train_img_files)

test_img_chips, test_mask_chips, test_edge_chips = model.generate_test_dataset(test_img_files)

test_set = tf.data.Dataset.from_tensor_slices((test_img_chips, (test_mask_chips, test_edge_chips)))


In [ ]:
train_img_chips, (train_mask_chips, train_edge_chips) =  next(train_generator)

ind = 200
fig = plt.figure(figsize=(15, 10), dpi=80)
fig.subplots_adjust(hspace=0.1, wspace=0.1)
ax = fig.add_subplot(2, 3, 1)
ax.imshow(train_img_chips)
ax = fig.add_subplot(2, 3, 2)
ax.imshow(train_mask_chips)
ax = fig.add_subplot(2, 3, 3)
ax.imshow(train_edge_chips)

- create the model and compile it

In [16]:
unet = model.get_do_unet()

unet.compile(optimizer="adam",
             loss=model.focal_loss,
             loss_weights=[0.3, 0.7],
             metrics=["acc"])

unet.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 188, 188, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_45 (Conv2D)             (None, 186, 186, 96  2688        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_54 (BatchN  (None, 186, 186, 96  384        ['conv2d_45[0][0]']              
 ormalization)                  )                                                           

- Train the model

In [17]:
_batch_size = 8
_epochs = 4
_workers = 8
_model_name = 'focal_loss'
images_per_epoch = 1000
train_steps = images_per_epoch//_batch_size
validation_steps = len(test_img_files)//_batch_size

unet.fit(train_set.batch(_batch_size), 
         epochs=_epochs, 
         validation_data=test_set.batch(_batch_size), 
         steps_per_epoch=train_steps,
         max_queue_size=2*_workers,
         use_multiprocessing=_useMultiProcessing,
         workers=_workers,
         verbose=1,
         callbacks=model.get_callbacks(_model_name))

Epoch 1/4
125/125 [==============================] - ETA: 0s - loss: 0.4406 - mask_loss: 0.3153 - edge_loss: 0.4943 - mask_acc: 0.4055 - edge_acc: 0.0617

In [ ]:
img = test_img_chips[np.random.randint(0, len(test_img_chips), 1)[0]]

img = np.array([np.squeeze(img)])


prediction = unet.predict(img)

np.array(prediction).min()


In [ ]:
np.random.randint(0, 5, 1)[0]